# Desenvolver o codigo para automatizar a busca no Rec Aq

* https://www.reclameaqui.com.br/empresa/bet365/lista-reclamacoes/?pagina=1

In [1]:
from bs4 import BeautifulSoup
from unidecode import unidecode

import pandas as pd
import numpy as np

import string
import requests

In [2]:
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}

def get_complains(company_id, company_name, n_complains):

    complains = {'company_name': [], 'description': [], 'href': [], 'full_complain': []}
    
    for n in range(0, n_complains, 10):
        url = f'https://iosearch.reclameaqui.com.br/raichu-io-site-search-v1/query/companyComplains/10/{n}'
        res = requests.get(url, params={'company': company_id}, headers=HEADERS).json()

        for x in range(10):
            data = res['complainResult']['complains']['data']
            
            description = data[x]['description']
            soup = BeautifulSoup(description, 'lxml').text
            complains['description'].append(soup)
            
            id_ = data[x]['id']
            title = data[x]['title']
            clean_title = title.translate(str.maketrans('', '', string.punctuation))
            href = clean_title.lower().strip() + '_' + id_
            href = unidecode(href.replace(' ', '-'))
            complains['href'].append(href)
            
            complains['company_name'].append(company_name)
            
            complains['full_complain'].append('')
            
    df = pd.DataFrame(complains)
    
    for i, company_name, href in zip(df.index, df.company_name, df.href):
        try:
            url = f"https://www.reclameaqui.com.br/{company_name}/{href}/"
            res = requests.get(url, headers=HEADERS)

            soup = BeautifulSoup(res.text, 'lxml')
            complain = soup.find('p', {'class': 'lzlu7c-17 fXwQIB'}).text
            df.iloc[i]['full_complain'] = complain
        except:
            df.drop(index=i, inplace=True)

            
    return df

In [3]:
empresas = pd.read_csv('../../data/empresas-ids.csv')
empresas

,id,empresa,segmento
0,76804,bet365,apostas
1,18598,sportingbet,apostas
2,GY8NGYlobI4FPcTZ,betano,apostas
3,106278,betfair,apostas
4,RpZ3Ktc2B1Ng2iHD,pixbet,apostas
5,knIrmsHcDugES1br,estrelabet,apostas
6,90DsVv_IWVkgr5GE,betnacional,apostas
7,MOByJdwgRX-4Kk0A,kto-sportsbook-betting-casino-online,apostas
8,Azy0j6BlGo95ApKe,marjosports,apostas
9,n_ATccuVQozJrDcd,betway,apostas


In [4]:
run = empresas[empresas.segmento == 'gaming_grande']
run

,id,empresa,segmento
15,36990,steam,gaming_grande


In [5]:
for company_id, company_name in zip(run.id, run.empresa):
    try:
        df = get_complains(company_id, company_name, 1010)
        df.to_csv(f'../data/{company_name}.csv')
        print(f'Finalizado {company_name}')
    except:
        try:
            df = get_complains(company_id, company_name, 510)
            df.to_csv(f'../data/{company_name}.csv')
            print(f'Finalizado {company_name}') 
        except:
            print(f'Erro na {company_name}')
            continue
    

/home/vitor/.pyenv/versions/3.10.6/envs/estudo/lib/python3.10/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


Finalizado steam
